<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/system_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchain_experimental >> requirements.txt
!echo openai >> requirements.txt
!echo anthropic >> requirements.txt
!echo cohere >> requirements.txt
!echo streamlit==1.29.0 >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo pypdf >> requirements.txt
!echo unstructured >> requirements.txt
!echo networkx >> requirements.txt
!echo openpyxl >> requirements.txt
!echo rapidocr-onnxruntime >> requirements.txt

In [ ]:
pip install -r requirements.txt

In [22]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [21]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

login(token = '---')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
#raghelper.py
%%writefile raghelper.py
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS # it is vector database. we will use FAISS database. It is created by Facebook.
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import os
from dotenv import load_dotenv

#load_dotenv()
#my_key_openai = os.getenv("openai_apikey")
#my_key_google = os.getenv("google_apikey")
#my_key_cohere = os.getenv("cohere_apikey")
#my_key_hf = os.getenv("huggingface_access_token")
my_key_openai="---"
my_key_google=""
my_key_cohere="----"
my_key_hf="---"

from langchain_community.llms import CTransformers

llm_llama = CTransformers(model="meta-llama/Meta-Llama-3-8B")

#llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model="gemini-pro")

embeddings = OpenAIEmbeddings(api_key=my_key_openai)
# embeddings = CohereEmbeddings(cohere_api_key=my_key_cohere, model="embed-multilingual-v3.0") #embed-english-v3.0

#embeddings = HuggingFaceInferenceAPIEmbeddings(
#    api_key=my_key_hf,
#    model_name="sentence-transformers/all-MiniLM-l6-v2"
#)

def ask_llama(prompt):
  AI_Response = llm_llama.invoke(prompt)

  return AI_Response

def rag_with_url(target_url,prompt):
  #load url
  loader = WebBaseLoader(target_url)
  raw_documents = loader.load() # raw_documents is document list (list[document])

  #create chunk using splitters
  #below code can run without any parameters but to reach the best answer you should try different parameters.
  #  text_splitter = RecursiveCharacterTextSplitter()
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=0,
      length_function=len # it provides that how to estimate lenght of chunk. we will use len because of python list rule.
  )

  # we run split_documents method which is inside of text_splitter and save to splitted_document
  splitted_document = text_splitter.split_documents(raw_documents)

  # we need to save splited documents in vector database.
  # FAISS create vectors from our documents using our embedding model and save vectordatabase.
  vectorstore = FAISS.from_documents(splitted_document, embeddings)
  # we reach as_retriever method in vectorstore. We convert vector store into retriever.
  # retriever won't retrieve all vectors, it retrieves vector considering context similarity among vector and prompt.
  #mostly, similarity is estimated by cosine similarity.
  retriever = vectorstore.as_retriever()
  relevant_documents = retriever.get_relevant_documents(prompt) # it retrieves the most similar document accordingly our prompt. It can be 3,4 or more documents. Default value is 4.

  context_data =""

  #Also, we use build function instead of the below code.
  for document in relevant_documents:
    context_data = context_data + " " + document.page_content # document has two features which are metada and page_content.
    #because the main data is in context_data.
  final_prompt = f"""Şöyle bir sorum var: {prompt}
  Bu soruyu yanıtlamak için elimizde şu bilgiler var: {context_data} .
  Bu sorunun yanıtını vermek için yalnızca sana burada verdiğim eldeki bilgileri kullan. Bunların dışına asla çıkma.
  """
  AI_Response = llm_llama.invoke(prompt)

  return AI_Response

def rag_with_pdf(filepath, prompt):

    loader = PyPDFLoader(filepath)

    raw_documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0,
        length_function=len
    )


    splitted_documents = text_splitter.split_documents(raw_documents)

    vectorstore = FAISS.from_documents(splitted_documents, embeddings)
    retriever = vectorstore.as_retriever()

    relevant_documents = retriever.get_relevant_documents(prompt)

    context_data = ""

    for document in relevant_documents:
        context_data = context_data + " " + document.page_content

    final_prompt = f"""Şöyle bir sorum var: {prompt}
    Bu soruyu yanıtlamak için elimizde şu bilgiler var: {context_data} .
    Bu sorunun yanıtını vermek için yalnızca sana burada verdiğim eldeki bilgileri kullan. Bunların dışına asla çıkma.
    """

    AI_Response = llm_llama.invoke(final_prompt)

    return AI_Response, relevant_documents

Overwriting raghelper.py


In [17]:
%%writefile rag.py
#Streamlit
import streamlit as st
import raghelper

st.set_page_config(page_title="Langchain ile Bellek Genişletme",layout="wide")
st.title("Langchain ile Bellek Genişletme: URL")
st.divider()

col_input, col_rag, col_normal = st.columns([1,2,2])

with col_input:
  target_url = st.text_input(label="İşlenecek Web Adresinizi Giriniz:")
  st.divider()
  prompt=st.text_input(label="Sorunuzu Giriniz:", key="url_prompt")
  st.divider()
  submit_btn = st.button(label="Sor", key="url_button")
  st.divider()

  if submit_btn:
    with col_rag:
      with st.spinner("Yanıt Hazırlanıyor...."):
        st.success("YANIT - Bellek Genişletme Devrede")
        st.markdown(raghelper.rag_with_url(target_url=target_url,prompt=prompt))
        st.divider()

    with col_normal:
      with st.spinner("Yanıt Hazırlanıyor...."):
        st.info("YANIT - Bellek Genişletme Devre Dışı") # info provides blue
        st.markdown(raghelper.ask_llama(prompt=prompt))
        st.divider()

st.title("Langchain ile Bellek Genişletme: PDF")
st.divider()

col_input, col_rag, col_normal = st.columns([1,2,2])

with col_input:
    selected_file = st.file_uploader(label="İşlenecek Dosyayı Seçiniz", type="pdf")
    st.divider()
    prompt = st.text_input(label="Sorunuzu Giriniz:", key="pdf_prompt")
    st.divider()
    submit_btn = st.button(label="Sor", key="pdf_button")
    st.divider()


if submit_btn:
  with col_rag:
    with st.spinner("Yanıt Hazırlanıyor..."):
      st.success("YANIT - Bellek Genişletme Devrede")
      #AI_Response, relevant_documents = raghelper.rag_with_pdf(filepath=f"content/drive/MyDrive/{selected_file.name}", prompt=prompt) # we cannot upload document because of streamlit error on google colab but you can write this code in your local computer.
      AI_Response, relevant_documents = raghelper.rag_with_pdf(filepath ="/content/drive/MyDrive/gelecek.pdf", prompt=prompt)
      st.markdown(AI_Response)
      st.divider()
      for doc in relevant_documents:
        st.caption(doc.page_content)
        st.markdown(f"Kaynak: {doc.metadata}")
        st.divider()

  with col_normal:
    with st.spinner("Yanıt Hazırlanıyor..."):
      st.info("YANIT - Bellek Genişletme Devre Dışı")
      st.markdown(raghelper.ask_llama(prompt=prompt))
      st.divider()


Overwriting rag.py


In [20]:
!npm install localtunnel
!streamlit run /content/rag.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.225s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 2.148s
your url is: https://fine-peas-sleep.loca.lt
^C


In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch

In [30]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16}
)
messages = [
    {"role": "system", "content": "Answer questions"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):],end='.')

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text based on the input I receive. I can be used to create chatbots, virtual assistants, and other applications that require natural language understanding and generation capabilities. I'm here to help answer your questions, provide information, and even engage in a fun conversation!.